adaptation from https://www.kaggle.com/raghuramr/mnist-by-beginner-with-fastai-resnet34

this notebook shows how we can adapt dataframe to images so that it can used as vision dataset

In [ ]:
from fastai.vision.all import *

### loading csv and converting it into img
https://www.kaggle.com/raghuramr/mnist-by-beginner-with-fastai-resnet34

In [ ]:
def get_images(df):
    IMG_WIDTH =28
    IMG_HEIGHT=28
    
    df['img'] = df[df.columns[df.columns.str.startswith('pixel')]].apply(
        lambda x: PILImage(Image.fromarray(np.array(x.values).reshape((IMG_WIDTH,IMG_HEIGHT)).astype(np.uint8))),
        axis=1
    )
    
    return df[df.columns[[not x for x in df.columns.str.startswith('pixel')]]]

In [ ]:
path = Path("../input/sign-language-mnist")

In [ ]:
train_df = pd.read_csv(path/"sign_mnist_train"/"sign_mnist_train.csv")

In [ ]:
train_df = get_images(train_df)

In [ ]:
train_df.head()

the datablock

In [ ]:
def get_x(r): return r['img']
def get_y(r): return r['label']
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock), get_x = get_x, get_y=get_y)
dls = dblock.dataloaders(train_df)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate).to_fp16()
learn.fine_tune(20)

inference

In [ ]:
test_df = pd.read_csv("../input/sign-language-mnist/sign_mnist_test.csv")
test_df['ImageId'] = test_df.index + 1
test_df = get_images(test_df)
test_df.head()

In [ ]:
dl = learn.dls.test_dl(list(test_df['img']))
inp, preds, _, dec_preds = learn.get_preds(dl=dl, with_input=True,with_decoded=True)
test_df['Label'] = dec_preds

In [ ]:
submission = test_df[['ImageId', 'Label']]
submission.to_csv("submission.csv", index=False)